In [1]:
from scipy import sparse
import numpy as np
import json
from scipy.sparse import vstack

In [2]:
c3rLDA_nzwc = np.load("./model_persistence/all_eths_less_than_15000_1000_its_25_topics/nzwc_ORIGINAL.npy")

In [3]:
c3rLDA_nzwc.shape

(25, 822218, 28)

In [3]:
c3rLDA_nwc = np.sum(c3rLDA_nzwc, axis=0)

In [4]:
c3rLDA_nwc.shape

(822218, 28)

In [26]:
c3rLDA_nwc = (c3rLDA_nwc + .01).astype(float)
c3rLDA_nwc /= np.sum(c3rLDA_nwc, axis=0)[np.newaxis, :]

In [38]:
mex_fre_nwc = c3rLDA_nwc[:, (18 ,25)]

In [9]:
mex_fre_nwc.shape

(822218, 2)

In [39]:
mex_fre_nwc = (mex_fre_nwc + .01).astype(float)
mex_fre_nwc /= np.sum(mex_fre_nwc, axis=0)[np.newaxis, :]

## Find locations

In [5]:
X_val = sparse.load_npz('/Users/tingchen/Desktop/Yelp Project/data_n_20_removed_all_eths_less_than_15000/X_val.npz')
cc_val= np.load("/Users/tingchen/Desktop/Yelp Project/data_n_20_removed_all_eths_less_than_15000/y_val.npy")
locs_val = []
with open("/Users/tingchen/Desktop/Yelp Project/data_n_20_removed_all_eths_less_than_15000/locs_val.json", 'r') as file:
    locs_val = json.load(file)

In [6]:
X_test = sparse.load_npz('/Users/tingchen/Desktop/Yelp Project/data_n_20_removed_all_eths_less_than_15000/X_test.npz')
cc_test= np.load("/Users/tingchen/Desktop/Yelp Project/data_n_20_removed_all_eths_less_than_15000/y_test.npy")
locs_test = []
with open("/Users/tingchen/Desktop/Yelp Project/data_n_20_removed_all_eths_less_than_15000/locs_test.json", 'r') as file:
    locs_test = json.load(file)

In [7]:
total_X = sparse.vstack([X_val, X_test])
total_c = np.concatenate((cc_val, cc_test))
total_locs = locs_val + locs_test

In [8]:
ix = total_X.getnnz(1)>0
total_X = total_X[ix]
total_c= total_c[ix]
total_locs= np.array(total_locs)[ix]

In [9]:
total_c.shape

(1430699,)

In [10]:
total_X

<1430699x822218 sparse matrix of type '<class 'numpy.float64'>'
	with 64269838 stored elements in Compressed Sparse Row format>

In [17]:
total_locs.shape

(1430699, 2)

In [44]:
sw_us_coords = ((36.377, -118.072), (30.687, -105.051))
se_ca_coords = ((47.020, -81.473), (42.825, -72.608))

sw_us_ids = []
se_ca_ids = []

for i in range(total_locs.shape[0]):

    
    loc_x = total_locs[i, 0]
    loc_y = total_locs[i, 1]
    
    if ((loc_x < sw_us_coords[0][0]) and (loc_x > sw_us_coords[1][0])):

        if (loc_y > sw_us_coords[0][1] and loc_y < sw_us_coords[1][1]):
            
            sw_us_ids.append(i)
            
    if (loc_x < se_ca_coords[0][0] and loc_x > se_ca_coords[1][0]):
        if (loc_y > se_ca_coords[0][1] and loc_y < se_ca_coords[1][1]):
            se_ca_ids.append(i)

In [30]:
len(se_ca_ids)

224230

In [45]:
sw_us_X = total_X[sw_us_ids]
se_ca_X = total_X[se_ca_ids]
se_ca_c = total_c[se_ca_ids]
sw_us_c = total_c[sw_us_ids]

In [46]:
sw_us_reviews = [[]  for i in range(28)]
se_ca_reviews = [[]  for i in range(28)]

In [47]:
for i in range(se_ca_c.shape[0]):
    se_ca_reviews[se_ca_c[i]].append(se_ca_X[i])

In [40]:
for i in range(28):
    print(len(se_ca_reviews[i]))

27824
28014
8998
590
19907
743
1444
483
3348
4188
15855
26125
3480
1109
9965
11826
10326
1274
8937
1360
9947
1476
3047
811
8182
12073
1124
1774


In [48]:
for i in range(sw_us_c.shape[0]):
    sw_us_reviews[sw_us_c[i]].append(sw_us_X[i])

In [42]:
for i in range(28):
    print(len(sw_us_reviews[i]))

0
70531
11410
4989
92243
15548
5740
5110
4224
16716
184713
51960
5022
3915
11319
184343
20841
2292
114459
10009
27941
14180
3302
1813
18271
17472
4677
5084


## Calculate Scores

In [16]:
table_sw_us = [sw_us_reviews[6], sw_us_reviews[16], sw_us_reviews[1], sw_us_reviews[8], sw_us_reviews[9]]

In [17]:
table_se_ca = [se_ca_reviews[6], se_ca_reviews[16], se_ca_reviews[1], se_ca_reviews[8], se_ca_reviews[9]]

In [9]:
from numpy import ma

In [ ]:
table_sw_us_scores = []
table_se_ca_scores = []

for i, cuisine in enumerate(table_sw_us_arr):
    cuisine_score = []
    print("\rCuisine {}".format(i), end="")
    for j, review in enumerate(cuisine):
        if j % 1000 == 0:
            print("\rReview {}".format(j), end="")
        mex_score = np.sum(ma.log(np.multiply(review, mex_fre_nwc[:, 0])).filled(0))
        fre_score = np.sum(ma.log(np.multiply(review, mex_fre_nwc[:, 1])).filled(0))
        #print((mex_score, fre_score))
        cuisine_score.append((mex_score, fre_score))
    table_sw_us_scores.append(cuisine_score) 

Review 1000

#### classification with argmax of probs

In [14]:
c3rLDA_nwc.shape

(822218, 28)

In [37]:
log_c3rLDA_nwc = np.log(c3rLDA_nwc)

In [62]:
np.argmax(np.dot(total_X[999].toarray(), log_c3rLDA_nwc))

1

In [64]:
np.dot(total_X[999].toarray(), log_c3rLDA_nwc)

array([[-1367.78952749, -1163.52214182, -1389.81801638, -1337.20034183,
        -1223.85133471, -1339.31885872, -1366.92590663, -1341.83149205,
        -1344.46892516, -1397.14703977, -1278.80212285, -1228.42036496,
        -1376.48267479, -1304.25783284, -1334.47585088, -1287.46508567,
        -1312.65399171, -1322.79921622, -1277.02305248, -1396.7396324 ,
        -1324.88404994, -1355.117783  , -1308.36938678, -1316.14699459,
        -1309.42814554, -1398.87291533, -1323.86901126, -1335.45411868]])

In [66]:
total_c[999]

11

In [67]:
#cuisine_scores = np.empty((total_X.shape[0], c3rLDA_nwc.shape[1]))
predictions = np.empty((10001, ))

for j, review in enumerate(total_X[:10000]):
    if j % 1000 == 0:
        print("\rReview {}".format(j), end="")
    
    predictions[j] = np.argmax(np.dot(total_X[j].toarray(), log_c3rLDA_nwc))

Review 9000

In [68]:
from sklearn.metrics import classification_report

In [78]:
print(classification_report(total_c[:10000], predictions[:10000]))

              precision    recall  f1-score   support

           0       0.20      0.01      0.02       228
           1       0.28      0.74      0.41       837
           2       0.39      0.12      0.18       170
           3       0.12      0.04      0.06        48
           4       0.31      0.75      0.44       980
           5       0.40      0.04      0.07       111
           6       0.20      0.04      0.06        52
           7       0.29      0.04      0.08        45
           8       0.17      0.03      0.05        61
           9       0.21      0.02      0.03       190
          10       0.46      0.05      0.09      1878
          11       0.41      0.25      0.31       678
          12       0.40      0.03      0.05        75
          13       0.10      0.09      0.09        56
          14       0.54      0.44      0.48       173
          15       0.37      0.06      0.11      1843
          16       0.40      0.06      0.11       229
          17       0.18    

#### classification for lda

In [ ]:
from tqdm import tqdm
from sklearn.decomposition

In [ ]:
cuisine_scores = []


for j, review in enumerate(total_X):
    if j % 1000 == 0:
        print("\rReview {}".format(j), end="")
    
    mex_score = np.sum(ma.log(np.multiply(review, mex_fre_nwc[:, 0])).filled(0))
    fre_score = np.sum(ma.log(np.multiply(review, mex_fre_nwc[:, 1])).filled(0))
    #print((mex_score, fre_score))
    cuisine_score.append((mex_score, fre_score))

In [21]:
base = sparse.load_npz("se_ca_table_4_base.npz")

In [20]:
del base

In [21]:
base = base[:5000]

In [22]:
base.shape

(4188, 822218)

In [53]:
a = np.array(cuisine_score)

In [54]:
a.shape

(5000, 2)

In [49]:
len(se_ca_reviews[25])

12073

In [55]:
np.save("french_se_ca_scores", a)

In [50]:
base = se_ca_reviews[25][0]
    
for j in range(1, 5000):
    base = vstack((base, se_ca_reviews[25][j]))
    


In [40]:
sparse.save_npz("se_ca_table_4_base", base)

## Do classification

Spanish

In [3]:
se_ca_spanish = np.load("se_ca_table_0_scores.npy")

In [4]:
sw_us_spanish = np.load("sw_us_table_0_scores.npy")

accuracy: sw us

In [17]:
1- np.sum(np.argmax(sw_us_spanish, axis = 1))/(sw_us_spanish.shape[0]) 

0.9625435540069687

accuracy: se ca

In [18]:
np.sum(np.argmax(se_ca_spanish, axis = 1))/(se_ca_spanish.shape[0]) 

0.036011080332409975

In [19]:
se_ca_spanish

array([[ -275.05400213,  -300.73056878],
       [-1409.09605489, -1559.17627142],
       [-1808.99069334, -1963.86928774],
       ...,
       [  -75.11933912,   -78.52224607],
       [ -483.49001211,  -578.20599238],
       [ -647.26729799,  -764.20238547]])

Korean

In [20]:
se_ca_korean = np.load("se_ca_table_1_scores.npy")
sw_us_korean = np.load("sw_us_table_1_scores.npy")

In [21]:
1- np.sum(np.argmax(sw_us_korean, axis = 1))/(sw_us_korean.shape[0])

0.989

In [22]:
np.sum(np.argmax(se_ca_korean, axis = 1))/(se_ca_korean.shape[0]) 

0.0092

Japanese

In [23]:
se_ca_japanese = np.load("se_ca_table_2_scores.npy")
sw_us_japanese = np.load("sw_us_table_2_scores.npy")

In [24]:
1- np.sum(np.argmax(sw_us_japanese, axis = 1))/(sw_us_japanese.shape[0])

0.9878

In [25]:
np.sum(np.argmax(se_ca_japanese, axis = 1))/(se_ca_japanese.shape[0]) 

0.0182

Pakistani

In [26]:
se_ca_pakistani = np.load("se_ca_table_3_scores.npy")
sw_us_pakistani = np.load("sw_us_table_3_scores.npy")

In [27]:
1- np.sum(np.argmax(sw_us_pakistani, axis = 1))/(sw_us_pakistani.shape[0])

0.9659090909090909

In [28]:
np.sum(np.argmax(se_ca_pakistani, axis = 1))/(se_ca_pakistani.shape[0])

0.028076463560334528

Greek

In [29]:
se_ca_greek = np.load("se_ca_table_4_scores.npy")
sw_us_greek = np.load("sw_us_table_4_scores.npy")

In [30]:
1- np.sum(np.argmax(sw_us_greek, axis = 1))/(sw_us_greek.shape[0])

0.9659090909090909

In [34]:
np.sum(np.argmax(se_ca_greek, axis = 1))/(se_ca_greek.shape[0])

0.026743075453677174

French

In [56]:
np.sum(np.argmax(a, axis = 1))/(a.shape[0])

0.1618

In [57]:
a

array([[ -156.78931253,  -176.60749076],
       [ -713.8585973 ,  -724.18979282],
       [ -378.15003511,  -387.35189877],
       ...,
       [-1205.74696551, -1312.31164237],
       [ -689.85008515,  -788.3223077 ],
       [ -395.28528203,  -405.94504872]])